In [1]:
## Imports ##

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Customer segmentation problem

Company X needs help to identify which customers to approach with a Software product offering. In order not to spend time and money on the wrong customers, they need to come up with a model that best predicts potential buyers. These types of models are essential in modern day businesses where companies need to find the right target audience. The company has access to a data set with information about previous customers and whether they bought the Software product or not. The dataset contains the following variables:

OrganisationNo - Customer Organisation Number.
<br>
LineDiscount - Customer Discount Agreement (Yes/No).
<br>
CustomerUnit - Customer Size Segmentation (Size of the company).
<br>
AccountManager - Name of the account manager.
<br>
Country - The country where the customer is located.
<br>
NoOfEmployees - Number of employees.
<br>
Potential - Customer IT spend potential.
<br>
AccessoryRatio - Share of accessories purchase.
<br>
NSB - Net sales (SEK).
<br>
CPLow - Customer profit (%).
<br>
Day - Days since last purchase.
<br>
TargetDays - Days since software purchase.
<br>
NoOfOrder - Number of orders.
<br>
Software - If the customer bought the software (Yes/No).
<br>
<br>
<br>
This is a binary classification problem where we want to predict wether a customer will buy a software product or not. We have labled data, and the column we want to predict is "Software". 

In [2]:
data = pd.read_excel('dataset.xlsx')
data.head(10)

,OrganisationNo,LineDiscount,CustomerUnit,AccountManager,Country,NoOfEmployees,Potential,AccessoryRatio,NSB,CPLow,Days,TargetDays,NrOfOrder,Software
0,1,No,SMB Small,Unassigned_SE,Sweden,1.3,0.0,1.000000,4399.00,25.113435,518,NaN,1,No
1,2,No,SMB Small,Not Defined,Finland,1.8,0.0,0.320656,873.24,19.260455,432,NaN,1,No
2,3,No,SMB Small,Unassigned_SE,Sweden,1.8,16956.0,0.053654,22272.00,13.261764,173,NaN,2,No
3,4,Yes,Corporate,Jan Tufvander,Sweden,49.9,448601.0,0.607573,44568.89,10.066057,4,NaN,41,No
4,5,No,SMB Mid,Unassigned_NO,Norway,68.0,0.0,0.000000,975.06,35.340389,704,NaN,1,No
5,6,No,SMB Small,Unassigned_SE,Sweden,1.8,0.0,0.000000,8194.00,10.240908,627,NaN,2,No
6,7,No,SMB Small,Unassigned_SE,Sweden,6.5,0.0,0.311970,36324.00,13.697555,589,NaN,7,No
7,8,No,SMB Small,Unassigned_SE,Sweden,1.3,0.0,0.000000,8999.00,4.162574,607,NaN,1,No
8,9,No,SMB Small,Unassigned_SE,Sweden,1.8,15102.0,0.077282,47203.64,8.445726,179,875.0,15,Yes
9,10,No,SMB Small,Unassigned_DK,Denmark,6.5,0.0,0.001645,53682.22,8.573137,482,482.0,4,Yes


# Exploratory Data Analysis

The dataset has 14 columns and 20 122 observations

In [3]:
data.shape

(20122, 14)

I check if there are any duplicates in the data.

In [4]:
data[data.duplicated()]

,OrganisationNo,LineDiscount,CustomerUnit,AccountManager,Country,NoOfEmployees,Potential,AccessoryRatio,NSB,CPLow,Days,TargetDays,NrOfOrder,Software


## Missing values

I will begin to explore if we have any missing values in the dataset.

In [5]:
data.isna().sum()

OrganisationNo        0
LineDiscount          0
CustomerUnit          1
AccountManager        0
Country               0
NoOfEmployees         0
Potential             0
AccessoryRatio        0
NSB                   0
CPLow                 0
Days                  0
TargetDays        16609
NrOfOrder             0
Software              0
dtype: int64

The column TargetDays is missing a majority of its values. TargetDays tells us how many days since the customer purchased the Software product, so if the customer never purcahsed the product this column will have a missing value. I will drop this column because it contains to many missing values and also because it will be directly correlated with the column we want to predict, "Software", since it will contain a numeric value if the customer bought the Software product and NA otherwise. 

The column CustomerUnit only has one missing value. I choose to drop that observation. Finally, I also choose to drop the column OrganisationNo since it is only a variable indicating what customer each observation belongs to.

In [6]:
data = data.drop(columns = ['OrganisationNo', 'TargetDays'])
data = data.dropna().reset_index(drop = True)

Now the datset has 12 columns and 20 121 observations.

In [7]:
data.shape

(20121, 12)

## Data types

I will now look at the different data types.

In [8]:
data.dtypes

LineDiscount       object
CustomerUnit       object
AccountManager     object
Country            object
NoOfEmployees     float64
Potential         float64
AccessoryRatio    float64
NSB               float64
CPLow             float64
Days                int64
NrOfOrder           int64
Software           object
dtype: object

### Numeric variables

First, I will look at the numeric variables.

In [9]:
data.describe()

,NoOfEmployees,Potential,AccessoryRatio,NSB,CPLow,Days,NrOfOrder
count,20121.000000,2.012100e+04,20121.000000,2.012100e+04,2.012100e+04,20121.000000,20121.000000
mean,109.872715,1.041357e+06,0.284309,1.637452e+05,4.911784e+11,248.159336,17.990010
std,988.155128,9.739245e+06,0.331148,2.349407e+06,6.575739e+13,218.141672,129.767426
min,-1.000000,0.000000e+00,0.000000,1.820000e-12,5.290000e-03,1.000000,1.000000
25%,1.800000,0.000000e+00,0.019782,3.911710e+03,8.914708e+00,49.000000,1.000000
50%,1.800000,1.960200e+04,0.156450,1.274600e+04,1.236654e+01,195.000000,3.000000
75%,13.400000,1.262280e+05,0.396909,4.551000e+04,1.761320e+01,400.000000,8.000000
max,49464.100000,4.459925e+08,1.537065,2.811631e+08,9.310000e+15,1076.000000,11018.000000


The variable NoOfEmployees have values that is not an integer, and I believe it makes more sense that the number of employees of a company is an integer. Also, the variable NoOfEmplyees have the value -1 for some customers. I choose to round all values to the closest integer and I change all -1's to 0 (I suspect that -1 means that the customer has 0 employees). 

In [10]:
data['NoOfEmployees'] = data['NoOfEmployees'].round()
data['NoOfEmployees'] = data['NoOfEmployees'].replace(-1, 0)

The variable AccessoryRatio have values greater than 1 which does not make sense since it is a ratio. I choose to remove the observations that has a Accessory ration greater than 1.

In [11]:
data[data['AccessoryRatio'] > 1]

,LineDiscount,CustomerUnit,AccountManager,Country,NoOfEmployees,Potential,AccessoryRatio,NSB,CPLow,Days,NrOfOrder,Software
1014,No,SMB Small,Unassigned_NO,Norway,1.0,7280.0,1.007461,1931.37,34.509183,53,2,No
5428,No,SMB Small,Unassigned_NO,Norway,0.0,0.0,1.033137,221.80,78.106402,532,6,No
5676,No,SMB Small,Unassigned_DK,Denmark,2.0,0.0,1.001601,3121.39,10.180721,515,3,No
6872,No,SMB Small,Unassigned_DK,Denmark,2.0,0.0,1.038384,866.49,11.827026,410,7,No
7852,No,SMB Small,Unassigned_SE,Sweden,13.0,157182.0,1.036201,152.48,30.653200,126,2,No
8946,No,Public,Michael Haagen Petersen,Denmark,30.0,305400.0,1.004659,19736.46,31.887583,200,5,No
13856,No,SMB Mid,Unassigned_DK,Denmark,43.0,1138509.0,1.058904,2215.79,24.245529,246,6,No
15250,Yes,Corporate,Lasse Pettersen,Norway,279.0,0.0,1.537065,6030.27,19.271111,414,3,No
17633,No,SMB Small,Unassigned_DK,Denmark,1.0,0.0,1.010070,3054.60,11.268251,580,3,No


In [12]:
data = data[data['AccessoryRatio'] <=1].reset_index(drop = True)

For all numeric variables we have some extreme values, these are values that are much larger than the median. In this dataset it makes sense that some customers have much larger values for NoOfEmployees, Potential, NSB and NrOfOrder since these depend on the size of the company. For example, looking at some customers with more than 2 million in net sales, they all have hight values in NoOfEmployees, Potetial and NrOfOrder.

In [13]:
data[data['NSB'] > 2000000].head(5)

,LineDiscount,CustomerUnit,AccountManager,Country,NoOfEmployees,Potential,AccessoryRatio,NSB,CPLow,Days,NrOfOrder,Software
58,Yes,Public,Matias Lohtander,Finland,932.0,894184.02,0.154561,4595239.07,4.665007,5,17,No
76,Yes,Public,Rasmus Winther,Denmark,1656.0,19361310.00,0.245075,8248682.32,4.345662,12,342,Yes
185,Yes,Public,Erik Markusson,Sweden,7611.0,68674149.97,0.073211,42934480.52,1.957098,1,1721,Yes
433,Yes,Public,Lasse Pettersen,Norway,2556.0,21012465.00,0.162918,2241556.43,2.696011,1,122,No
458,Yes,Corporate,Henrik Åqvist,Norway,1840.0,22590288.00,0.140020,25009054.49,5.312397,1,2522,Yes


For CPLow, I found some extreme values that I choose to remove.

In [14]:
data = data[data['CPLow'] < 1000].reset_index(drop = True)

### Categorical variables

Now, I will look at the categorical variables.

In [15]:
print('Categories for LineDiscount:', data['LineDiscount'].unique())
print('\n')
print('Categories for CustomerUnit:', data['CustomerUnit'].unique())
print('\n')
print('Categories for Country:', data['Country'].unique())
print('\n')
print('Categories for Software:', data['Software'].unique())

Categories for LineDiscount: ['No' 'Yes']


Categories for CustomerUnit: ['SMB Small' 'Corporate' 'SMB Mid' 'SMB Large' 'Public' 'Home'
 'Not Defined' 'IC - Intercompany']


Categories for Country: ['Sweden' 'Finland' 'Norway' 'Denmark']


Categories for Software: ['No' 'Yes']


AccountManager has 202 different categories. I will redefine this variable to "Yes" if the customer has an account manager and "No" otherwise. I will do this to reduce the number of categories but still keep the information if the customer has an account manager or not. 

In [16]:
data['AccountManager'].value_counts()

Unassigned_SE            10233
Unassigned_DK             3496
Unassigned_NO             2069
Mid - Segmentet            949
Not Defined                579
                         ...  
Mats Audas                   1
Liv Janne Berg               1
Bente Grønberg Møller        1
Pablo Gonzalez               1
Andreas Lantz                1
Name: AccountManager, Length: 202, dtype: int64

In [17]:
data = data.replace({'AccountManager' : {'Unassigned_SE' : 'No', 'Unassigned_DK' : 'No', 'Unassigned_NO' : 'No',
                                        'Not Defined' : 'No'}})
data.loc[data['AccountManager'] != 'No', 'AccountManager'] = 'Yes'

In [18]:
print('Categories for AccountManager:', data['AccountManager'].unique())

Categories for AccountManager: ['No' 'Yes']


In order to keep the categorical values in the data set that I will use to train the machine learning model I need to translate them into numeric variables. I will do this by creating dummy variables. I will make dummy variables for all categorical variables except Software since this is the varibale we want to predict.

In [19]:
dummy_data = pd.get_dummies(data[['LineDiscount', 'CustomerUnit', 'Country', 'AccountManager']], drop_first = False)

In [20]:
dummy_data.head()

,LineDiscount_No,LineDiscount_Yes,CustomerUnit_Corporate,CustomerUnit_Home,CustomerUnit_IC - Intercompany,CustomerUnit_Not Defined,CustomerUnit_Public,CustomerUnit_SMB Large,CustomerUnit_SMB Mid,CustomerUnit_SMB Small,Country_Denmark,Country_Finland,Country_Norway,Country_Sweden,AccountManager_No,AccountManager_Yes
0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0
1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
2,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0
3,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0


Now, I will concat the numerical variables and the dummy variables to the final data set.

In [21]:
numeric_data = data[['Software','NoOfEmployees', 'Potential', 'AccessoryRatio', 'NSB', 'Days', 'NrOfOrder']]
final_data = pd.concat([numeric_data, dummy_data], axis = 1)

In [22]:
final_data.head()

,Software,NoOfEmployees,Potential,AccessoryRatio,NSB,Days,NrOfOrder,LineDiscount_No,LineDiscount_Yes,CustomerUnit_Corporate,...,CustomerUnit_Public,CustomerUnit_SMB Large,CustomerUnit_SMB Mid,CustomerUnit_SMB Small,Country_Denmark,Country_Finland,Country_Norway,Country_Sweden,AccountManager_No,AccountManager_Yes
0,No,1.0,0.0,1.000000,4399.00,518,1,1,0,0,...,0,0,0,1,0,0,0,1,1,0
1,No,2.0,0.0,0.320656,873.24,432,1,1,0,0,...,0,0,0,1,0,1,0,0,1,0
2,No,2.0,16956.0,0.053654,22272.00,173,2,1,0,0,...,0,0,0,1,0,0,0,1,1,0
3,No,50.0,448601.0,0.607573,44568.89,4,41,0,1,1,...,0,0,0,0,0,0,0,1,0,1
4,No,68.0,0.0,0.000000,975.06,704,1,1,0,0,...,0,0,1,0,0,0,1,0,1,0


The final data set has 20 107 observations and 23 columns.

In [23]:
final_data.shape

(20107, 23)

# Training models

After exploring and preparing the data, I now have a final data set I will use for modeling. 

When using machine learning models for classification it is important that the dataset is balanced. That is, the dataset contains, in this case, approximately the same amount of customers that bought the software product and customer that did not buy the software product. If that is the case, one could just split the dataset into a training and test set and go ahead with modeling. However, this data set is unbalanced, approximately 80% of the data points belongs to customer that did not buy the software product. Therefore it is important to balance the classes in the training data. This is done in order for the model to learn about the minority class which in this case would be the customers that did buy the product. To balance the data, I will use Synthetic Minority Over-Sampling Technique (SMOTE). SMOTE generates synthetic data points based on the original data where the minority class is oversampled. The advantage with SMOTE is that the data points are not duplicated, instead you are creating synthetic data points that are slightly different from the original data points.

## Train test split

I will begin by randomly splitting the data set into a training set and a test set. The training set will contain 70% of the samples and the test set would contain 30% of the samples. I will then use SMOTE to balance the training data.

In [24]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Train test split
y = final_data['Software']
X = final_data.drop(columns = 'Software')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 101)

# SMOTE
smote = SMOTE(random_state = 101)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [25]:
y_train_smote.value_counts()

No     11623
Yes    11623
Name: Software, dtype: int64

## Fitting models

Next step is to fit the training set to some selected models. In this case I would begin with fitting the training set to a Logistic Regression, a Random Forest Classifier and a Support Vector Machines. The test set will be saved and used to evaluate the performance of the models. 

I will begin to fit “baseline” models using the default values of the hyperparameters given by the functions. I will evaluate the performance of the models looking at the accuracy and classification report.

In [53]:
# Logistic regression
from sklearn.linear_model import LogisticRegression

log_clf = LogisticRegression()
log_clf.fit(X_train_smote, y_train_smote)
log_pred = log_clf.predict(X_test)

# Random forest classification
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(random_state = 101)
rf_clf.fit(X_train_smote, y_train_smote)
rf_pred = rf_clf.predict(X_test)

# Support vector machines
from sklearn.svm import SVC

svm_clf = SVC(random_state = 101)
svm_clf.fit(X_train_smote, y_train_smote)
svm_pred = svm_clf.predict(X_test)

In [54]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Metrics Logistic regression
print(f"Accuracy of the logistic regression is: {round(accuracy_score(y_test, log_pred), 2)}")
print(f"Classification report for logistic regression:\n {classification_report(y_test, log_pred)}")
print(f"Confusion matrix for logistic regression:\n {confusion_matrix(y_test, log_pred)}")
print('\n')

# Metrics Random forest classifier
print(f"Accuracy of the random forest classifier is: {round(accuracy_score(y_test, rf_pred), 2)}")
print(f"Classification report for random forest classifier:\n {classification_report(y_test, rf_pred)}")
print(f"Confusion matrix for randomforest classifier:\n {confusion_matrix(y_test, rf_pred)}")
print('\n')

# Metrics Support vector machine
print(f"Accuracy of the support vector machines is: {round(accuracy_score(y_test, svm_pred), 2)}")
print(f"Classification report for support vector machines:\n {classification_report(y_test, svm_pred)}")
print(f"Confusion matrix for support vector machines:\n {confusion_matrix(y_test, svm_pred)}")

Accuracy of the logistic regression is: 0.19
Classification report for logistic regression:
               precision    recall  f1-score   support

          No       0.92      0.02      0.04      5012
         Yes       0.17      0.99      0.29      1021

    accuracy                           0.19      6033
   macro avg       0.54      0.51      0.17      6033
weighted avg       0.79      0.19      0.08      6033

Confusion matrix for logistic regression:
 [[ 108 4904]
 [  10 1011]]


Accuracy of the random forest classifier is: 0.81
Classification report for random forest classifier:
               precision    recall  f1-score   support

          No       0.89      0.89      0.89      5012
         Yes       0.45      0.44      0.44      1021

    accuracy                           0.81      6033
   macro avg       0.67      0.66      0.67      6033
weighted avg       0.81      0.81      0.81      6033

Confusion matrix for randomforest classifier:
 [[4463  549]
 [ 573  448]]


Ac

The logistic regression model performed very poorly. It is possible that this model is too "simple" for this problem. Both the random forest and SVM have a accuracy over 0.8 which is good. However, the precision, recall and f1-score is not so good fo both models for the "Yes" class. We can see in the confusion matrix that both the random forest and SVM have more false negatives than true negatives, that is the models predicts 'No' for the 'Yes' class more often than it actually predicts correct for the 'Yes' class.

## Grid search

To optimize the performance of the models I will tune the hyperparameters using GridSearchCV from Scikit-learn. GridSearchCV is the process of performing hyperparameter tuning in order to determine the optimal values for a given model. GridSearchCV trains the models using different combinations of values of the hyperparameters and will return the model with the best performance. This method uses Cross-Validation when training the models. Tuning hyperparameters and evaluating the model only on the training set can lead to overfitting. That is, the model performs really well on the training set but will not be able to generalize on new data, such as the test set. Using cross-validation is a common way of dealing with overfitting when tuning the hyperparameters. 

I choose to do the grid search on the random forest since I beleive it will perform the best among the baseline models.

Using grid search to find the hyperparameters that results in the best performing model is a time consuming task as you have to fit a model for each combination of the hyperparameter values specified in the parameter grid. Therefore, I choose to perform the grid search only on a random sample from the training set to speed up the process.

In [55]:
training_set = pd.concat([y_train_smote, X_train_smote], axis = 1)
training_sample = training_set.sample(2000)

X_train_sample = training_sample.drop(columns = 'Software')
y_train_sample = training_sample['Software']

In [56]:
from sklearn.model_selection import GridSearchCV

rf_clf = RandomForestClassifier(random_state = 101)

parameter_grid = {'n_estimators': [100, 500, 1000], 
              'max_depth': [100, 500, None],
              'min_samples_split': [2]}

gridsearch_rf_clf = GridSearchCV(estimator = rf_clf, param_grid = parameter_grid, n_jobs = -1)
gridsearch_rf_clf.fit(X_train_sample, y_train_sample)

GridSearchCV(estimator=RandomForestClassifier(random_state=101), n_jobs=-1,
             param_grid={'max_depth': [100, 500, None],
                         'min_samples_split': [2],
                         'n_estimators': [100, 500, 1000]})

In [57]:
print('Best parameters:', gridsearch_rf_clf.best_params_)

Best parameters: {'max_depth': 100, 'min_samples_split': 2, 'n_estimators': 1000}


I tried different combinations of values for the hyperparameters and selected the model with the best performance. In a setting where you have more time to explore different combinations of the hyperparameter values, I would add more hyperparameters to the parameter grid. For this task I am satisfied with the values chosen by the grid search as I noticed that the performance of the model did not change much depending on the values of the hyperparameters.

## Evaluation

I fit a random forest classifier with the parameter values provided by the grid search.

In [58]:
rf_clf = RandomForestClassifier(random_state = 101, n_estimators = 1000, max_depth = 100, min_samples_split = 2)
rf_clf.fit(X_train_smote, y_train_smote)
rf_pred = rf_clf.predict(X_test)

# Metrics Random forest
print(f"Accuracy of the random forest is: {round(accuracy_score(y_test, rf_pred), 2)}")
print(f"Classification report for random forest:\n {classification_report(y_test, rf_pred)}")
print(f"Confusion matrix for random forest:\n {confusion_matrix(y_test, rf_pred)}")

Accuracy of the random forest is: 0.81
Classification report for random forest:
               precision    recall  f1-score   support

          No       0.89      0.89      0.89      5012
         Yes       0.44      0.44      0.44      1021

    accuracy                           0.81      6033
   macro avg       0.66      0.66      0.66      6033
weighted avg       0.81      0.81      0.81      6033

Confusion matrix for random forest:
 [[4446  566]
 [ 570  451]]


The model did not perform much better with the new values of the hyperparametes. The accuracy of the model is good, however there is still some problem with the precision, recall and f1-score for the 'Yes'-class. In reality this means that company X will miss out on potential buyers since alot of 'Yes'-customers are predicted as 'No'.